### **1. Chuẩn bị tập dữ liệu**

#### 1.1 Import thư viện

In [33]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
import graphviz
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo
from IPython.display import Image, display
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os

#### 1.2 Xử lý dữ liệu

In [ ]:
# Tải dữ liệu UCI Heart Disease từ ucimlrepo
heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
y = heart_disease.data.targets.squeeze()  # Loại bỏ cột đơn nếu tồn tại

# Kết hợp lại thành một DataFrame đầy đủ
df = pd.concat([X, y.rename("target")], axis=1)

# Lưu ra file CSV
df.to_csv("heart_disease.csv", index=False)

print("Đã lưu dữ liệu thành file heart_disease.csv")
# Chuyển y thành nhị phân: 0 = không bệnh, 1 = có bệnh
y_binary = y.copy()
y_binary[y_binary > 0] = 1

df_binary = df.copy()
df_binary['target'] = y_binary
display(df_binary)

#### 1.3 Chia dữ liệu

In [35]:
def stratified_split(X, y, train_size, random_state=42):
    """
    Chia dữ liệu theo kiểu phân tầng.
    
    Parameters:
    - X: Dữ liệu đầu vào (features)
    - y: Nhãn (labels)
    - test_size: Tỷ lệ dữ liệu test
    - random_state: Giá trị seed để tái tạo
    
    Returns:
    - feature_train, feature_test, label_train, label_test: Dữ liệu sau khi chia
    """
    test_size = 1 - train_size
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )
    return X_train, X_test, y_train, y_test

In [36]:
def visualize_train_test_distribution(y, y_train, y_test, train_size):
    fig, axes = plt.subplots(1, 3, figsize=(15, 7), sharey=True)
    
    label = [0, 1]
    
    for ax in axes:
        ax.set_xticks([0, 1])  # Chỉ hiển thị 0 và 1 trên trục x
        ax.set_xlim(-0.5, 1.5)  # Giữ khoảng trục x cố định
    
    # Phân phối gốc
    original_counts = [Counter(y)[ele] for ele in label]
    axes[0].bar(label, original_counts)
    axes[0].set_title('Original Dataset')
    axes[0].set_ylabel('Count')
    for i, count in enumerate(original_counts):
        axes[0].text(i, count + 1, str(count), ha='center', va='bottom', fontsize=15)

    # Phân phối tập train
    train_counts = [Counter(y_train)[ele] for ele in label]
    axes[1].bar(label, train_counts)
    axes[1].set_title(f'Training Set ({int(round(train_size, 2)* 100)}%)')
    for i, count in enumerate(train_counts):
        axes[1].text(i, count + 1, str(count), ha='center', va='bottom', fontsize=15)

    # Phân phối tập test
    test_counts = [Counter(y_test)[ele] for ele in label]
    axes[2].bar(label, test_counts)
    axes[2].set_title(f'Test Set ({int(round((1 - train_size), 2) * 100)}%)')
    for i, count in enumerate(test_counts):
        axes[2].text(i, count + 1, str(count), ha='center', va='bottom', fontsize=15)

    plt.tight_layout()
    os.makedirs('./output/split', exist_ok=True)
    plt.savefig(f'./output/split/train_test_{int(train_size*100)}_{int(round(1 - train_size, 2) * 100)}.png', format='png', bbox_inches='tight')
    plt.show()


##### Visualize Data sau khi chia tỷ lệ

In [ ]:
ratios = [0.4, 0.6, 0.8, 0.9]

split_train_test = {}  # Dùng để lưu các tập sau khi chia
for train_size in ratios:
    feature_train, feature_test, label_train, label_test = stratified_split(X,y_binary, train_size, random_state=42)
    visualize_train_test_distribution(y_binary, label_train, label_test, train_size)
    split_train_test[train_size] = {
        'feature_train': feature_train,
        'label_train': label_train,
        'feature_test': feature_test,
        'label_test': label_test
    }

### **2. Xây dựng Decision Tree**

In [ ]:
# ratios = [0.4, 0.6, 0.8, 0.9]
y_trains, y_tests = [], []

for ratio in ratios:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_binary, train_size=ratio, stratify=y_binary, shuffle=True, random_state=42)

    y_trains.append(y_train)
    y_tests.append(y_test)

    # Huấn luyện cây quyết định
    clf = DecisionTreeClassifier(criterion='entropy', random_state=42)
    clf.fit(X_train, y_train)

    # Trực quan hóa cây
    dot_data = export_graphviz(clf, out_file=None,
                                feature_names=X.columns,
                                class_names=['No HD', 'HD'],
                                filled=True, rounded=True,
                                special_characters=True)
    graph = graphviz.Source(dot_data)
    graph.render(f"./output/decision_tree_{int(ratio*100)}", format='png', cleanup=True)
    display(Image(f"./output/decision_tree_{int(ratio*100)}.png"))


### **3. Đánh giá cây quyết định**

In [ ]:
for ratio in ratios:
    # Chia lại dữ liệu với y_binary (phân lớp nhị phân)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_binary, train_size=ratio, stratify=y_binary, shuffle=True, random_state=42)

    # Huấn luyện mô hình cây quyết định
    clf = DecisionTreeClassifier(criterion='entropy', random_state=42)
    clf.fit(X_train, y_train)

    # Dự đoán trên tập test
    y_pred = clf.predict(X_test)

    # In báo cáo phân loại
    print(f"\n==== Classification Report (Train/Test = {round(ratio*100)}/{round((1-ratio)*100)}) ====")
    print(classification_report(y_test, y_pred, target_names=['No HD', 'HD']))

    # Tính ma trận nhầm lẫn
    cm = confusion_matrix(y_test, y_pred)

    # Vẽ ma trận nhầm lẫn
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No HD', 'HD'], yticklabels=['No HD', 'HD'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix (Train/Test = {round(ratio*100)}/{round((1-ratio)*100)})')
    plt.tight_layout()
    plt.show()

### **4 Độ sâu và độ chính xác của cây quyết định**

In [ ]:
# Chia dữ liệu 80/20 với y_binary
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, train_size=0.8, stratify=y_binary, shuffle=True, random_state=42)

max_depth_values = [None, 2, 3, 4, 5, 6, 7]
accuracies = []

for depth in max_depth_values:
    clf = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=42)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Trực quan hóa cây
    dot_data = export_graphviz(clf, out_file=None,
                               feature_names=X.columns,
                               class_names=['No HD', 'HD'],
                               filled=True, rounded=True,
                               special_characters=True)
    graph = graphviz.Source(dot_data)
    depth_label = "full" if depth is None else str(depth)
    graph.render(f"./output/decision_tree_maxdepth_{depth_label}", format='png', cleanup=True)
    display(Image(f"./output/decision_tree_maxdepth_{depth_label}.png"))


#### Bảng đánh giá và biểu đồ

In [ ]:
# In bảng accuracy
print("\nAccuracy scores for different max_depth values:")
print("Max Depth\tAccuracy")
for depth, acc in zip(max_depth_values, accuracies):
    label = "Full Tree" if depth is None else str(depth)
    print(f"{label:<10}\t{acc * 100:.2f}%") 

# Vẽ biểu đồ
labels = ['Full' if d is None else str(d) for d in max_depth_values]

plt.figure(figsize=(8, 5))
# Chuyển accuracies sang phần trăm
accuracies_percent = [acc * 100 for acc in accuracies]
plt.plot(labels, accuracies_percent, marker='o', linestyle='-', color='blue')
plt.ylabel("Accuracy on Test Set (%)")
plt.xlabel("Max Depth")
plt.ylabel("Accuracy on Test Set")
plt.grid(True)
plt.tight_layout()
plt.show()